In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql

In [2]:
# connect to SQLite database
db_path = "../Data/basketball.sqlite"
connection = sql.connect(db_path) # create connection object to database
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", connection)
print(table)

                     name
0                  Player
1                    Team
2         Team_Attributes
3            Team_History
4       Player_Attributes
5          Game_Officials
6   Game_Inactive_Players
7             Team_Salary
8           Player_Salary
9                   Draft
10          Draft_Combine
11          Player_Photos
12            Player_Bios
13                   Game
14                   News
15           News_Missing


In [3]:
biosQ = """SELECT * FROM 'Player_Attributes'"""

bios_df = pd.read_sql(biosQ, connection)
bios_df.tail()

,ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PTS,AST,REB,ALL_STAR_APPEARANCES,PIE
4495,1627790,Ante,Zizic,Ante Zizic,"Zizic, Ante",A. Zizic,ante-zizic,1997-01-04T00:00:00,,Croatia,...,Y,Y,2016,1,23,6.0,0.6,3.9,0.0,NaN
4496,78647,Jim,Zoet,Jim Zoet,"Zoet, Jim",J. Zoet,jim-zoet,1953-12-30T00:00:00,Kent State,USA,...,Y,Y,Undrafted,Undrafted,Undrafted,0.3,0.1,1.1,0.0,NaN
4497,78648,Bill,Zopf,Bill Zopf,"Zopf, Bill",B. Zopf,bill-zopf,1948-06-07T00:00:00,Duquesne,USA,...,Y,Y,1970,2,33,2.2,1.4,0.9,0.0,NaN
4498,1627826,Ivica,Zubac,Ivica Zubac,"Zubac, Ivica",I. Zubac,ivica-zubac,1997-03-18T00:00:00,Mega Basket,Croatia,...,Y,Y,2016,2,32,8.5,1.1,7.0,NaN,0.126
4499,78650,Matt,Zunic,Matt Zunic,"Zunic, Matt",M. Zunic,matt-zunic,1919-12-19T00:00:00,George Washington,USA,...,Y,Y,1947,None,None,4.9,0.9,NaN,0.0,NaN


In [4]:
injuries_df = pd.read_csv('../Data/Injuries - KAggle.csv')
injuries_df.head()

,Key,Date,Team,Acquired,Relinquished,Notes
0,2010Carlos Boozer,10/3/2010,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010Jonas Jerebko,10/6/2010,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010Terrico White,10/6/2010,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010Jeff Ayres,10/8/2010,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010Troy Murphy,10/8/2010,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [5]:
injuries_df["Key"] = injuries_df["Key"].str.replace(".","")
injuries_df["Relinquished"] = injuries_df["Relinquished"].str.replace(".","")
injuries_df.head()

,Key,Date,Team,Acquired,Relinquished,Notes
0,2010Carlos Boozer,10/3/2010,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010Jonas Jerebko,10/6/2010,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010Terrico White,10/6/2010,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010Jeff Ayres,10/8/2010,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010Troy Murphy,10/8/2010,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [6]:
injuries_pivot = pd.pivot_table(injuries_df, values='Notes', index=['Key', 'Team','Relinquished'],
                    aggfunc='count')
injuries_pivot.head()

,,,Notes
Key,Team,Relinquished,
2010AJ Price,Pacers,AJ Price,4
2010Aaron Brooks,Rockets,Aaron Brooks,1
2010Aaron Gray,Hornets,Aaron Gray,5
2010Acie Law,Grizzlies,Acie Law,1
2010Al Harrington,Nuggets,Al Harrington,2


In [7]:
injuries_pivot.reset_index(inplace=True)

In [8]:
injuries_pivot.head()

,Key,Team,Relinquished,Notes
0,2010AJ Price,Pacers,AJ Price,4
1,2010Aaron Brooks,Rockets,Aaron Brooks,1
2,2010Aaron Gray,Hornets,Aaron Gray,5
3,2010Acie Law,Grizzlies,Acie Law,1
4,2010Al Harrington,Nuggets,Al Harrington,2


In [9]:
team_Q = """SELECT * FROM 'Team'"""

teams_df = pd.read_sql(team_Q, connection)
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [10]:
teams_df.columns

Index(['id', 'full_name', 'abbreviation', 'nickname', 'city', 'state',
       'year_founded'],
      dtype='object')

In [11]:
injuries_team=pd.merge(injuries_pivot,teams_df[['id','full_name','abbreviation','nickname']],how="left",left_on='Team',right_on='nickname').fillna(np.nan)
injuries_team.head()

,Key,Team,Relinquished,Notes,id,full_name,abbreviation,nickname
0,2010AJ Price,Pacers,AJ Price,4,1610612754,Indiana Pacers,IND,Pacers
1,2010Aaron Brooks,Rockets,Aaron Brooks,1,1610612745,Houston Rockets,HOU,Rockets
2,2010Aaron Gray,Hornets,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,Hornets
3,2010Acie Law,Grizzlies,Acie Law,1,1610612763,Memphis Grizzlies,MEM,Grizzlies
4,2010Al Harrington,Nuggets,Al Harrington,2,1610612743,Denver Nuggets,DEN,Nuggets


In [12]:
injuries_pivot.shape

(5071, 4)

In [13]:
injuries_team.shape

(5071, 8)

In [14]:
injuries_team=injuries_team.drop(['Team','nickname'],axis=1)

In [15]:
injuries_team.head()

,Key,Relinquished,Notes,id,full_name,abbreviation
0,2010AJ Price,AJ Price,4,1610612754,Indiana Pacers,IND
1,2010Aaron Brooks,Aaron Brooks,1,1610612745,Houston Rockets,HOU
2,2010Aaron Gray,Aaron Gray,5,1610612766,Charlotte Hornets,CHA
3,2010Acie Law,Acie Law,1,1610612763,Memphis Grizzlies,MEM
4,2010Al Harrington,Al Harrington,2,1610612743,Denver Nuggets,DEN


In [16]:
injuries_team["NewKey"]=injuries_team["Key"]+injuries_team["abbreviation"]
injuries_team.head()

,Key,Relinquished,Notes,id,full_name,abbreviation,NewKey
0,2010AJ Price,AJ Price,4,1610612754,Indiana Pacers,IND,2010AJ PriceIND
1,2010Aaron Brooks,Aaron Brooks,1,1610612745,Houston Rockets,HOU,2010Aaron BrooksHOU
2,2010Aaron Gray,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,2010Aaron GrayCHA
3,2010Acie Law,Acie Law,1,1610612763,Memphis Grizzlies,MEM,2010Acie LawMEM
4,2010Al Harrington,Al Harrington,2,1610612743,Denver Nuggets,DEN,2010Al HarringtonDEN


In [17]:
injuries_team=injuries_team.drop(['Key'],axis=1)
injuries_team.head()

,Relinquished,Notes,id,full_name,abbreviation,NewKey
0,AJ Price,4,1610612754,Indiana Pacers,IND,2010AJ PriceIND
1,Aaron Brooks,1,1610612745,Houston Rockets,HOU,2010Aaron BrooksHOU
2,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,2010Aaron GrayCHA
3,Acie Law,1,1610612763,Memphis Grizzlies,MEM,2010Acie LawMEM
4,Al Harrington,2,1610612743,Denver Nuggets,DEN,2010Al HarringtonDEN


In [18]:
injuries_team=injuries_team.rename(columns={"Relinquished": "Player", "Notes": "Injuries","full_name":"TeamName","abbreviation":"TeamCode",
                             "NewKey":"Key","id": "TeamId"})
injuries_team.head()

,Player,Injuries,TeamId,TeamName,TeamCode,Key
0,AJ Price,4,1610612754,Indiana Pacers,IND,2010AJ PriceIND
1,Aaron Brooks,1,1610612745,Houston Rockets,HOU,2010Aaron BrooksHOU
2,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,2010Aaron GrayCHA
3,Acie Law,1,1610612763,Memphis Grizzlies,MEM,2010Acie LawMEM
4,Al Harrington,2,1610612743,Denver Nuggets,DEN,2010Al HarringtonDEN


In [19]:
injuries_team['DraftYear']=np.nan
injuries_team.head()

,Player,Injuries,TeamId,TeamName,TeamCode,Key,DraftYear
0,AJ Price,4,1610612754,Indiana Pacers,IND,2010AJ PriceIND,NaN
1,Aaron Brooks,1,1610612745,Houston Rockets,HOU,2010Aaron BrooksHOU,NaN
2,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,2010Aaron GrayCHA,NaN
3,Acie Law,1,1610612763,Memphis Grizzlies,MEM,2010Acie LawMEM,NaN
4,Al Harrington,2,1610612743,Denver Nuggets,DEN,2010Al HarringtonDEN,NaN


In [20]:
# capturing Injury year from key
injuries_team['DraftYear'] = injuries_team['Key'].str[:4]
injuries_team.head()    

,Player,Injuries,TeamId,TeamName,TeamCode,Key,DraftYear
0,AJ Price,4,1610612754,Indiana Pacers,IND,2010AJ PriceIND,2010
1,Aaron Brooks,1,1610612745,Houston Rockets,HOU,2010Aaron BrooksHOU,2010
2,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,2010Aaron GrayCHA,2010
3,Acie Law,1,1610612763,Memphis Grizzlies,MEM,2010Acie LawMEM,2010
4,Al Harrington,2,1610612743,Denver Nuggets,DEN,2010Al HarringtonDEN,2010


In [21]:
playersQ = """SELECT * FROM 'Player'"""

players_df = pd.read_sql(playersQ, connection)
players_df.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,0
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,0
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,0
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,0
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,0


In [22]:
injuries_team_player = injuries_team.merge(players_df[['id','full_name']],how='left',left_on='Player',right_on='full_name').fillna(np.nan)
injuries_team_player.head()

,Player,Injuries,TeamId,TeamName,TeamCode,Key,DraftYear,id,full_name
0,AJ Price,4,1610612754,Indiana Pacers,IND,2010AJ PriceIND,2010,201985,AJ Price
1,Aaron Brooks,1,1610612745,Houston Rockets,HOU,2010Aaron BrooksHOU,2010,201166,Aaron Brooks
2,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,2010Aaron GrayCHA,2010,201189,Aaron Gray
3,Acie Law,1,1610612763,Memphis Grizzlies,MEM,2010Acie LawMEM,2010,201151,Acie Law
4,Al Harrington,2,1610612743,Denver Nuggets,DEN,2010Al HarringtonDEN,2010,1733,Al Harrington


In [23]:
injuries_team_player.shape

(5079, 9)

In [24]:
injuries_team_player=injuries_team_player.drop(['full_name'],axis=1)
injuries_team_player=injuries_team_player.rename(columns={"id": "PlayerId"})

In [25]:
injuries_team_player=injuries_team_player.drop(['Key'],axis=1)

In [26]:
injuries_team_player.head()

,Player,Injuries,TeamId,TeamName,TeamCode,DraftYear,PlayerId
0,AJ Price,4,1610612754,Indiana Pacers,IND,2010,201985
1,Aaron Brooks,1,1610612745,Houston Rockets,HOU,2010,201166
2,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,2010,201189
3,Acie Law,1,1610612763,Memphis Grizzlies,MEM,2010,201151
4,Al Harrington,2,1610612743,Denver Nuggets,DEN,2010,1733


## Bring in All Seasons data to get points

In [27]:
seasons_df = pd.read_csv('../Data/AllSeasons.csv')
seasons_df.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Travis Knight,LAL,22,213.36,106.59412,Connecticut,USA,1996,1,29,...,4.8,4.5,0.5,6.2,0.127,0.182,0.142,0.536,0.052,1996-97
1,Matt Fish,MIA,27,210.82,106.59412,North Carolina-Wilmington,USA,1996,2,50,...,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97
2,Matt Bullard,HOU,30,208.28,106.59412,Iowa,USA,1996,Undrafted,Undrafted,...,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97
3,Marty Conlon,BOS,29,210.82,111.13004,Providence,USA,1996,Undrafted,Undrafted,...,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97
4,Martin Muursepp,DAL,22,205.74,106.59412,None,USA,1996,1,25,...,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97


In [28]:
seasons_df["player_name"] = seasons_df["player_name"].str.replace(".","")

In [29]:
seasons_df_clean = seasons_df[seasons_df['draft_year'] != 'Undrafted']
seasons_df_clean.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Travis Knight,LAL,22,213.36,106.59412,Connecticut,USA,1996,1,29,...,4.8,4.5,0.5,6.2,0.127,0.182,0.142,0.536,0.052,1996-97
1,Matt Fish,MIA,27,210.82,106.59412,North Carolina-Wilmington,USA,1996,2,50,...,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97
2,Matt Bullard,HOU,30,208.28,106.59412,Iowa,USA,1996,Undrafted,Undrafted,...,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97
3,Marty Conlon,BOS,29,210.82,111.13004,Providence,USA,1996,Undrafted,Undrafted,...,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97
4,Martin Muursepp,DAL,22,205.74,106.59412,None,USA,1996,1,25,...,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97


In [30]:
seasons_df_clean = seasons_df_clean[seasons_df_clean['draft_year'] >= 2010]
seasons_df_clean.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
6226,Darrell Arthur,MEM,23,205.74,106.594120,Kansas,USA,2010,1,27,...,9.1,4.3,0.7,2.0,0.082,0.167,0.216,0.534,0.056,2010-11
6227,AJ Price,IND,24,187.96,82.100152,Connecticut,USA,2010,2,52,...,6.5,1.4,2.2,1.6,0.022,0.083,0.227,0.454,0.233,2010-11
6228,Maurice Evans,WAS,32,195.58,99.790240,Texas,USA,2010,Undrafted,Undrafted,...,6.3,2.1,0.6,-7.8,0.036,0.080,0.140,0.513,0.046,2010-11
6229,Maurice Ager,MIN,27,195.58,91.625584,Michigan State,USA,2010,1,28,...,3.8,0.5,0.3,-12.9,0.000,0.095,0.216,0.682,0.063,2010-11
6230,Matt Carroll,CHA,30,198.12,96.161504,Notre Dame,USA,2010,Undrafted,Undrafted,...,4.4,1.3,0.4,-6.4,0.023,0.124,0.194,0.530,0.068,2010-11


In [31]:
seasons_df_clean['draft_year'] = seasons_df_clean['draft_year'].astype(str)

In [32]:
max(seasons_df_clean['draft_year'])

'2020'

In [33]:
seasons_df_clean['Key']= seasons_df_clean['player_name'] + seasons_df_clean['team_abbreviation']+seasons_df_clean['draft_year']
seasons_df_clean.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,Key
6226,Darrell Arthur,MEM,23,205.74,106.594120,Kansas,USA,2010,1,27,...,4.3,0.7,2.0,0.082,0.167,0.216,0.534,0.056,2010-11,Darrell ArthurMEM2010
6227,AJ Price,IND,24,187.96,82.100152,Connecticut,USA,2010,2,52,...,1.4,2.2,1.6,0.022,0.083,0.227,0.454,0.233,2010-11,AJ PriceIND2010
6228,Maurice Evans,WAS,32,195.58,99.790240,Texas,USA,2010,Undrafted,Undrafted,...,2.1,0.6,-7.8,0.036,0.080,0.140,0.513,0.046,2010-11,Maurice EvansWAS2010
6229,Maurice Ager,MIN,27,195.58,91.625584,Michigan State,USA,2010,1,28,...,0.5,0.3,-12.9,0.000,0.095,0.216,0.682,0.063,2010-11,Maurice AgerMIN2010
6230,Matt Carroll,CHA,30,198.12,96.161504,Notre Dame,USA,2010,Undrafted,Undrafted,...,1.3,0.4,-6.4,0.023,0.124,0.194,0.530,0.068,2010-11,Matt CarrollCHA2010


In [34]:
injuries_team_player['Key'] = injuries_team_player['Player']+injuries_team_player['TeamCode']+injuries_team_player['DraftYear']
injuries_team_player.head()

,Player,Injuries,TeamId,TeamName,TeamCode,DraftYear,PlayerId,Key
0,AJ Price,4,1610612754,Indiana Pacers,IND,2010,201985,AJ PriceIND2010
1,Aaron Brooks,1,1610612745,Houston Rockets,HOU,2010,201166,Aaron BrooksHOU2010
2,Aaron Gray,5,1610612766,Charlotte Hornets,CHA,2010,201189,Aaron GrayCHA2010
3,Acie Law,1,1610612763,Memphis Grizzlies,MEM,2010,201151,Acie LawMEM2010
4,Al Harrington,2,1610612743,Denver Nuggets,DEN,2010,1733,Al HarringtonDEN2010


In [35]:
seasons_df_clean.columns

Index(['player_name', 'team_abbreviation', 'age', 'player_height',
       'player_weight', 'college', 'country', 'draft_year', 'draft_round',
       'draft_number', 'gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct',
       'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season', 'Key'],
      dtype='object')

In [36]:
injuries_team_player.columns

Index(['Player', 'Injuries', 'TeamId', 'TeamName', 'TeamCode', 'DraftYear',
       'PlayerId', 'Key'],
      dtype='object')

In [37]:
injuries_points = seasons_df_clean.merge(injuries_team_player[['Key','Injuries','PlayerId','TeamId']],how='left',on='Key').fillna(0)

In [38]:
injuries_points.shape

(5475, 25)

In [39]:
injuries_points.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,Key,Injuries,PlayerId,TeamId
0,Darrell Arthur,MEM,23,205.74,106.594120,Kansas,USA,2010,1,27,...,0.082,0.167,0.216,0.534,0.056,2010-11,Darrell ArthurMEM2010,1.0,201589,1610612763
1,AJ Price,IND,24,187.96,82.100152,Connecticut,USA,2010,2,52,...,0.022,0.083,0.227,0.454,0.233,2010-11,AJ PriceIND2010,4.0,201985,1610612754
2,Maurice Evans,WAS,32,195.58,99.790240,Texas,USA,2010,Undrafted,Undrafted,...,0.036,0.080,0.140,0.513,0.046,2010-11,Maurice EvansWAS2010,0.0,0,0
3,Maurice Ager,MIN,27,195.58,91.625584,Michigan State,USA,2010,1,28,...,0.000,0.095,0.216,0.682,0.063,2010-11,Maurice AgerMIN2010,2.0,200772,1610612750
4,Matt Carroll,CHA,30,198.12,96.161504,Notre Dame,USA,2010,Undrafted,Undrafted,...,0.023,0.124,0.194,0.530,0.068,2010-11,Matt CarrollCHA2010,0.0,0,0


In [40]:
injuries_points.columns

Index(['player_name', 'team_abbreviation', 'age', 'player_height',
       'player_weight', 'college', 'country', 'draft_year', 'draft_round',
       'draft_number', 'gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct',
       'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season', 'Key', 'Injuries',
       'PlayerId', 'TeamId'],
      dtype='object')

In [41]:
injuries_points=injuries_points.drop(['age', 'player_height', 'player_weight','college', 'country',
                       'draft_round','draft_number', 'net_rating', 'oreb_pct',
                       'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season'],axis=1)

In [42]:
injuries_points.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,0,0
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,0,0


In [43]:
injuries_points.shape

(5475, 11)

In [44]:
seasons_df_clean[seasons_df_clean['Key']=='Maurice EvansWAS2010']

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,Key
6228,Maurice Evans,WAS,32,195.58,99.79024,Texas,USA,2010,Undrafted,Undrafted,...,2.1,0.6,-7.8,0.036,0.08,0.14,0.513,0.046,2010-11,Maurice EvansWAS2010


## Update playerID and Team ID for missing ones

In [45]:
players_Q = """SELECT * FROM 'Player'"""

players_df = pd.read_sql(players_Q, connection)
players_df.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,0
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,0
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,0
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,0
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,0


In [46]:
players_df["full_name"] = players_df["full_name"].str.replace(".","")

In [47]:
players_df = players_df[players_df['id'].map(len)>=4]
players_df.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,0
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,0
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,0
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,0
6,76005,Tom Abernethy,Tom,Abernethy,0


In [48]:
players_df2=players_df[(players_df.duplicated(subset = ['full_name'])==True)]
players_df2.head()

,id,full_name,first_name,last_name,is_active
910,76519,Charlie Davis,Charlie,Davis,0
1066,76610,Bob Duffy,Bob,Duffy,0
1076,76616,Mike Dunleavy,Mike,Dunleavy,0
1554,76909,Matt Guokas,Matt,Guokas,0
1708,76990,Cedric Henderson,Cedric,Henderson,0


In [49]:
#players_df.to_csv('../Data/tokeep.csv',index=False)
players_df2.to_csv('../Data/todelete.csv',index=False)

In [50]:
#removed the duplicate players from the 2st file based on the 2nd file
players_df=pd.read_csv('../Data/tokeep.csv')
players_df.shape

(4017, 5)

In [51]:
player_injuries=injuries_points.merge(players_df[['full_name','id']],how='left',left_on='player_name',right_on='full_name').fillna(np.nan)
player_injuries.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId,full_name,id
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,Darrell Arthur,201589.0
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,AJ Price,201985.0
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,0,0,Maurice Evans,2230.0
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,Maurice Ager,200772.0
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,0,0,Matt Carroll,2679.0


In [52]:
player_injuries.shape

(5475, 13)

In [53]:
injuries_points.shape

(5475, 11)

In [54]:
player_injuries.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId,full_name,id
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,Darrell Arthur,201589.0
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,AJ Price,201985.0
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,0,0,Maurice Evans,2230.0
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,Maurice Ager,200772.0
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,0,0,Matt Carroll,2679.0


In [55]:
#drop full name and delete playerID with blank id, and then rename id to player id
clean_injuries1=player_injuries.drop(['full_name','PlayerId'],axis=1)
clean_injuries2=clean_injuries1.rename(columns = {'id':'PlayerId'})
clean_injuries2['PlayerId']=clean_injuries2['PlayerId'].astype(str)
clean_injuries2['PlayerId']=clean_injuries2['PlayerId'].str.replace("\.0","")
clean_injuries2.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,PlayerId
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,1610612763,201589
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,1610612754,201985
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,0,2230
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,1610612750,200772
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,0,2679


In [56]:
clean_injuries2.shape

(5475, 11)

In [57]:
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [58]:
clean_injuries=clean_injuries2.merge(teams_df[['id','abbreviation']],how="left",left_on='team_abbreviation',right_on='abbreviation')
clean_injuries.shape

(5475, 13)

In [59]:
clean_injuries.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,PlayerId,id,abbreviation
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,1610612763,201589,1610612763,MEM
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,1610612754,201985,1610612754,IND
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,0,2230,1610612764,WAS
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,1610612750,200772,1610612750,MIN
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,0,2679,1610612766,CHA


In [60]:
clean_injuries=clean_injuries.drop(['abbreviation','TeamId'],axis=1)
clean_injuries=clean_injuries.rename(columns = {'id':'TeamId'})
clean_injuries.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,2230,1610612764
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,2679,1610612766


In [61]:
clean_injuries.shape

(5475, 11)

In [62]:
min(clean_injuries['draft_year'])

'2010'

In [63]:
max(clean_injuries['draft_year'])

'2020'

## Get birthdate from player attributes table

In [64]:
playersAttr_Q = """SELECT * FROM 'Player_Attributes'"""

playersAttr_df = pd.read_sql(playersAttr_Q, connection)
playersAttr_df.head()

,ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PTS,AST,REB,ALL_STAR_APPEARANCES,PIE
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24T00:00:00,Duke,USA,...,Y,Y,1990,1,25,5.7,0.3,3.3,0.0,NaN
1,76002,Zaid,Abdul-Aziz,Zaid Abdul-Aziz,"Abdul-Aziz, Zaid",Z. Abdul-Aziz,zaid-abdul-aziz,1946-04-07T00:00:00,Iowa State,USA,...,Y,Y,1968,1,5,9.0,1.2,8.0,0.0,NaN
2,76003,Kareem,Abdul-Jabbar,Kareem Abdul-Jabbar,"Abdul-Jabbar, Kareem",K. Abdul-Jabbar,kareem-abdul-jabbar,1947-04-16T00:00:00,UCLA,USA,...,Y,Y,1969,1,1,24.6,3.6,11.2,18.0,NaN
3,51,Mahmoud,Abdul-Rauf,Mahmoud Abdul-Rauf,"Abdul-Rauf, Mahmoud",M. Abdul-Rauf,mahmoud-abdul-rauf,1969-03-09T00:00:00,Louisiana State,USA,...,Y,Y,1990,1,3,14.6,3.5,1.9,0.0,NaN
4,1505,Tariq,Abdul-Wahad,Tariq Abdul-Wahad,"Abdul-Wahad, Tariq",T. Abdul-Wahad,tariq-abdul-wahad,1974-11-03T00:00:00,San Jose State,France,...,Y,Y,1997,1,11,7.8,1.1,3.3,0.0,NaN


In [65]:
playersAttr_df.dtypes

ID                                   object
FIRST_NAME                           object
LAST_NAME                            object
DISPLAY_FIRST_LAST                   object
DISPLAY_LAST_COMMA_FIRST             object
DISPLAY_FI_LAST                      object
PLAYER_SLUG                          object
BIRTHDATE                            object
SCHOOL                               object
COUNTRY                              object
LAST_AFFILIATION                     object
HEIGHT                              float64
WEIGHT                              float64
SEASON_EXP                            int64
JERSEY                               object
POSITION                             object
ROSTERSTATUS                         object
GAMES_PLAYED_CURRENT_SEASON_FLAG     object
TEAM_ID                              object
TEAM_NAME                            object
TEAM_ABBREVIATION                    object
TEAM_CODE                            object
TEAM_CITY                       

In [66]:
injuries_all_up=clean_injuries.merge(playersAttr_df[['ID','BIRTHDATE']],how="left",left_on='PlayerId',right_on='ID')
injuries_all_up.shape

(5475, 13)

In [67]:
injuries_all_up.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId,ID,BIRTHDATE
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,201589,1988-03-25T00:00:00
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,201985,1986-10-07T00:00:00
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,2230,1610612764,2230,1978-11-08T00:00:00
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,200772,1984-02-09T00:00:00
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,2679,1610612766,2679,1980-08-28T00:00:00


In [68]:
injuries_all_up['BIRTHDATE']=pd.to_datetime(injuries_all_up['BIRTHDATE'])
injuries_all_up.dtypes

player_name                  object
team_abbreviation            object
draft_year                   object
gp                            int64
pts                         float64
reb                         float64
ast                         float64
Key                          object
Injuries                    float64
PlayerId                     object
TeamId                       object
ID                           object
BIRTHDATE            datetime64[ns]
dtype: object

In [69]:
from datetime import date
def calculateAge(birthDate):
    today = date.today()
    age = today.year - birthDate.year -((today.month, today.day) < (birthDate.month, birthDate.day))
 
    return age

In [70]:
for i,row in injuries_all_up.iterrows():
    age=calculateAge(row['BIRTHDATE'])
    injuries_all_up.loc[i,'Age']=age

In [71]:
injuries_all_up.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId,ID,BIRTHDATE,Age
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,201589,1988-03-25,33.0
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,201985,1986-10-07,34.0
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,2230,1610612764,2230,1978-11-08,42.0
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,200772,1984-02-09,37.0
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,2679,1610612766,2679,1980-08-28,41.0


In [72]:
injuries_all_up.shape

(5475, 14)

In [73]:
injuries_all_up.drop(['ID'],axis=1)

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId,BIRTHDATE,Age
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,1988-03-25,33.0
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,1986-10-07,34.0
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,2230,1610612764,1978-11-08,42.0
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,1984-02-09,37.0
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,2679,1610612766,1980-08-28,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5470,Matthew Dellavedova,CLE,2020,13,2.8,1.8,4.5,Matthew DellavedovaCLE2020,0.0,203521,1610612739,1990-09-08,31.0
5471,Maurice Harkless,SAC,2020,37,5.2,2.4,1.2,Maurice HarklessSAC2020,0.0,203090,1610612758,1993-05-11,28.0
5472,Max Strus,MIA,2020,39,6.1,1.1,0.6,Max StrusMIA2020,0.0,1629622,1610612748,1996-03-28,25.0
5473,Marcus Morris Sr,LAC,2020,57,13.4,4.1,1.0,Marcus Morris SrLAC2020,0.0,202694,1610612746,1989-09-02,32.0


In [74]:
injuries_all_up.to_csv("../Data/Injuries2010-2020.csv")

## code to update the missing birthdays

In [75]:
injuries_all_up.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId,ID,BIRTHDATE,Age
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,201589,1988-03-25,33.0
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,201985,1986-10-07,34.0
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,2230,1610612764,2230,1978-11-08,42.0
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,200772,1984-02-09,37.0
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,2679,1610612766,2679,1980-08-28,41.0


In [159]:
bday_miss=injuries_all_up[injuries_all_up['BIRTHDATE'].isnull()]
bday_miss.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId,ID,BIRTHDATE,Age
15,Marcus Camby,POR,2010,59,4.7,10.3,2.1,Marcus CambyPOR2010,0.0,nan,1610612757,NaN,NaT,NaN
58,Kevin Garnett,BOS,2010,71,14.9,8.9,2.4,Kevin GarnettBOS2010,1.0,nan,1610612738,NaN,NaT,NaN
63,Juwan Howard,MIA,2010,57,2.4,2.1,0.4,Juwan HowardMIA2010,3.0,nan,1610612748,NaN,NaT,NaN
106,Kurt Thomas,CHI,2010,52,4.1,5.8,1.2,Kurt ThomasCHI2010,4.0,nan,1610612741,NaN,NaT,NaN
109,Kobe Bryant,LAL,2010,82,25.3,5.1,4.7,Kobe BryantLAL2010,0.0,nan,1610612747,NaN,NaT,NaN


In [160]:
playersAttr_df.columns

Index(['ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'PTS', 'AST', 'REB',
       'ALL_STAR_APPEARANCES', 'PIE'],
      dtype='object')

In [161]:
bday_update = bday_miss.merge(playersAttr_df[['ID','DISPLAY_FIRST_LAST','BIRTHDATE']],how="left",left_on='player_name',right_on='DISPLAY_FIRST_LAST')
bday_update

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId,TeamId,ID_x,BIRTHDATE_x,Age,ID_y,DISPLAY_FIRST_LAST,BIRTHDATE_y
0,Marcus Camby,POR,2010,59,4.7,10.3,2.1,Marcus CambyPOR2010,0.0,nan,1610612757,NaN,NaT,NaN,948,Marcus Camby,1974-03-22T00:00:00
1,Kevin Garnett,BOS,2010,71,14.9,8.9,2.4,Kevin GarnettBOS2010,1.0,nan,1610612738,NaN,NaT,NaN,708,Kevin Garnett,1976-05-19T00:00:00
2,Juwan Howard,MIA,2010,57,2.4,2.1,0.4,Juwan HowardMIA2010,3.0,nan,1610612748,NaN,NaT,NaN,436,Juwan Howard,1973-02-07T00:00:00
3,Kurt Thomas,CHI,2010,52,4.1,5.8,1.2,Kurt ThomasCHI2010,4.0,nan,1610612741,NaN,NaT,NaN,703,Kurt Thomas,1972-10-04T00:00:00
4,Kobe Bryant,LAL,2010,82,25.3,5.1,4.7,Kobe BryantLAL2010,0.0,nan,1610612747,NaN,NaT,NaN,977,Kobe Bryant,1978-08-23T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Malik Fitts,LAC,2020,3,1.0,1.0,0.0,Malik FittsLAC2020,0.0,nan,1610612746,NaN,NaT,NaN,NaN,NaN,NaN
164,Mamadi Diakite,MIL,2020,14,3.1,2.4,0.6,Mamadi DiakiteMIL2020,0.0,nan,1610612749,NaN,NaT,NaN,NaN,NaN,NaN
165,Markus Howard,DEN,2020,37,2.8,0.6,0.5,Markus HowardDEN2020,0.0,nan,1610612743,NaN,NaT,NaN,NaN,NaN,NaN
166,Marques Bolden,CLE,2020,6,1.2,1.0,0.0,Marques BoldenCLE2020,0.0,nan,1610612739,NaN,NaT,NaN,NaN,NaN,NaN


In [162]:
bday_update['ID_x'].isna().sum()

168

In [163]:
bday_update.count()

player_name           168
team_abbreviation     168
draft_year            168
gp                    168
pts                   168
reb                   168
ast                   168
Key                   168
Injuries              168
PlayerId              168
TeamId                168
ID_x                    0
BIRTHDATE_x             0
Age                     0
ID_y                   56
DISPLAY_FIRST_LAST     56
BIRTHDATE_y            56
dtype: int64

In [164]:
bday_update=bday_update.drop(['PlayerId','DISPLAY_FIRST_LAST','ID_x','BIRTHDATE_x'],axis=1)

In [167]:
bday_update.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,Age,PlayerId,BIRTHDATE
0,Marcus Camby,POR,2010,59,4.7,10.3,2.1,Marcus CambyPOR2010,0.0,1610612757,NaN,948,1974-03-22T00:00:00
1,Kevin Garnett,BOS,2010,71,14.9,8.9,2.4,Kevin GarnettBOS2010,1.0,1610612738,NaN,708,1976-05-19T00:00:00
2,Juwan Howard,MIA,2010,57,2.4,2.1,0.4,Juwan HowardMIA2010,3.0,1610612748,NaN,436,1973-02-07T00:00:00
3,Kurt Thomas,CHI,2010,52,4.1,5.8,1.2,Kurt ThomasCHI2010,4.0,1610612741,NaN,703,1972-10-04T00:00:00
4,Kobe Bryant,LAL,2010,82,25.3,5.1,4.7,Kobe BryantLAL2010,0.0,1610612747,NaN,977,1978-08-23T00:00:00


In [166]:
bday_update=bday_update.rename(columns = {'ID_y':'PlayerId','BIRTHDATE_y':'BIRTHDATE'})

In [168]:
bday_update[bday_update['PlayerId'].isnull()]

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,Age,PlayerId,BIRTHDATE
26,Marcus Morris,HOU,2011,17,2.4,0.9,0.2,Marcus MorrisHOU2011,0.0,1610612745,NaN,NaN,NaN
35,Marcus Morris,PHX,2012,77,7.7,3.6,0.8,Marcus MorrisPHX2012,0.0,1610612756,NaN,NaN,NaN
52,Marcus Morris,PHX,2013,82,9.7,3.9,1.1,Marcus MorrisPHX2013,0.0,1610612756,NaN,NaN,NaN
56,Marcus Morris,PHX,2014,81,10.4,4.8,1.6,Marcus MorrisPHX2014,0.0,1610612756,NaN,NaN,NaN
59,Marcus Morris,DET,2015,80,14.1,5.1,2.5,Marcus MorrisDET2015,0.0,1610612765,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Malik Fitts,LAC,2020,3,1.0,1.0,0.0,Malik FittsLAC2020,0.0,1610612746,NaN,NaN,NaN
164,Mamadi Diakite,MIL,2020,14,3.1,2.4,0.6,Mamadi DiakiteMIL2020,0.0,1610612749,NaN,NaN,NaN
165,Markus Howard,DEN,2020,37,2.8,0.6,0.5,Markus HowardDEN2020,0.0,1610612743,NaN,NaN,NaN
166,Marques Bolden,CLE,2020,6,1.2,1.0,0.0,Marques BoldenCLE2020,0.0,1610612739,NaN,NaN,NaN


In [169]:
playersAttr_df[playersAttr_df['DISPLAY_FIRST_LAST']=='Charlie Brown Jr']

,ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PTS,AST,REB,ALL_STAR_APPEARANCES,PIE


In [188]:
bday_update.loc[bday_update['player_name']=='Danuel House','PlayerId']='1627863'

In [181]:
bday_update[bday_update['player_name']=='Marcus Morris']

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,Age,PlayerId,BIRTHDATE_y
0,Marcus Morris,HOU,2011,17,2.4,0.9,0.2,Marcus MorrisHOU2011,0.0,1610612745,NaN,202694,NaN
1,Marcus Morris,PHX,2012,77,7.7,3.6,0.8,Marcus MorrisPHX2012,0.0,1610612756,NaN,202694,NaN
2,Marcus Morris,PHX,2013,82,9.7,3.9,1.1,Marcus MorrisPHX2013,0.0,1610612756,NaN,202694,NaN
3,Marcus Morris,PHX,2014,81,10.4,4.8,1.6,Marcus MorrisPHX2014,0.0,1610612756,NaN,202694,NaN
4,Marcus Morris,DET,2015,80,14.1,5.1,2.5,Marcus MorrisDET2015,0.0,1610612765,NaN,202694,NaN
5,Marcus Morris,DET,2016,79,14.0,4.6,2.0,Marcus MorrisDET2016,3.0,1610612765,NaN,202694,NaN


In [190]:
bday_update.loc[bday_update['PlayerId']=='1627863','BIRTHDATE_y']='1993-06-07T00:00:00'

In [173]:
null_players = pd.read_csv('../Data/NullPlayers.csv')
null_players

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,PlayerId,BIRTHDATE
0,Donta Hall,BKN,2019,9,4.3,4.2,0.4,Donta HallBKN2019,0,1610612751,NaN,8/7/1997
1,Eric Mika,SAC,2019,1,6.0,7.0,0.0,Eric MikaSAC2019,0,1610612758,NaN,1/5/1995
2,Juan Toscano-Anderson,GSW,2019,13,5.3,4.0,2.0,Juan Toscano-AndersonGSW2019,0,1610612744,NaN,4/10/1993
3,Mychal Mulder,GSW,2019,7,11.0,3.3,1.1,Mychal MulderGSW2019,0,1610612744,NaN,6/12/1994
4,Malik Newman,CLE,2019,1,2.0,0.0,0.0,Malik NewmanCLE2019,0,1610612739,NaN,2/21/1997
...,...,...,...,...,...,...,...,...,...,...,...,...
98,Malik Fitts,LAC,2020,3,1.0,1.0,0.0,Malik FittsLAC2020,0,1610612746,NaN,7/4/1997
99,Mamadi Diakite,MIL,2020,14,3.1,2.4,0.6,Mamadi DiakiteMIL2020,0,1610612749,NaN,1/21/1997
100,Markus Howard,DEN,2020,37,2.8,0.6,0.5,Markus HowardDEN2020,0,1610612743,NaN,3/3/1999
101,Marques Bolden,CLE,2020,6,1.2,1.0,0.0,Marques BoldenCLE2020,0,1610612739,NaN,4/17/1998


In [174]:
bday_update=bday_update[bday_update['BIRTHDATE'].isnull()]
bday_update.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,Age,PlayerId,BIRTHDATE
26,Marcus Morris,HOU,2011,17,2.4,0.9,0.2,Marcus MorrisHOU2011,0.0,1610612745,NaN,NaN,NaN
35,Marcus Morris,PHX,2012,77,7.7,3.6,0.8,Marcus MorrisPHX2012,0.0,1610612756,NaN,NaN,NaN
52,Marcus Morris,PHX,2013,82,9.7,3.9,1.1,Marcus MorrisPHX2013,0.0,1610612756,NaN,NaN,NaN
56,Marcus Morris,PHX,2014,81,10.4,4.8,1.6,Marcus MorrisPHX2014,0.0,1610612756,NaN,NaN,NaN
59,Marcus Morris,DET,2015,80,14.1,5.1,2.5,Marcus MorrisDET2015,0.0,1610612765,NaN,NaN,NaN


In [175]:
bday_update = bday_update.merge(null_players[['player_name','BIRTHDATE']],how="left",left_on='player_name',right_on='player_name')
bday_update.tail()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,Age,PlayerId,BIRTHDATE_x,BIRTHDATE_y
113,Mamadi Diakite,MIL,2020,14,3.1,2.4,0.6,Mamadi DiakiteMIL2020,0.0,1610612749,NaN,NaN,NaN,1/21/1997
114,Markus Howard,DEN,2020,37,2.8,0.6,0.5,Markus HowardDEN2020,0.0,1610612743,NaN,NaN,NaN,3/3/1999
115,Marques Bolden,CLE,2020,6,1.2,1.0,0.0,Marques BoldenCLE2020,0.0,1610612739,NaN,NaN,NaN,4/17/1998
116,Marques Bolden,CLE,2020,6,1.2,1.0,0.0,Marques BoldenCLE2020,0.0,1610612739,NaN,NaN,NaN,4/17/1998
117,Mason Jones,PHI,2020,32,5.3,1.7,1.3,Mason JonesPHI2020,0.0,1610612755,NaN,NaN,NaN,7/21/1998


In [176]:
bday_update=bday_update.drop(['BIRTHDATE_x'],axis=1)
bday_update.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,Age,PlayerId,BIRTHDATE_y
0,Marcus Morris,HOU,2011,17,2.4,0.9,0.2,Marcus MorrisHOU2011,0.0,1610612745,NaN,NaN,NaN
1,Marcus Morris,PHX,2012,77,7.7,3.6,0.8,Marcus MorrisPHX2012,0.0,1610612756,NaN,NaN,NaN
2,Marcus Morris,PHX,2013,82,9.7,3.9,1.1,Marcus MorrisPHX2013,0.0,1610612756,NaN,NaN,NaN
3,Marcus Morris,PHX,2014,81,10.4,4.8,1.6,Marcus MorrisPHX2014,0.0,1610612756,NaN,NaN,NaN
4,Marcus Morris,DET,2015,80,14.1,5.1,2.5,Marcus MorrisDET2015,0.0,1610612765,NaN,NaN,NaN


In [177]:
bday_update.shape

(118, 13)

In [143]:
bday_update=bday_update.drop(['team_abbreviation_x', 'draft_year_x', 'gp_x', 'pts_x',
       'reb_x', 'ast_x', 'Key_x', 'Injuries_x', 'TeamId_x', 'PlayerId_x',
       'BIRTHDATE_x'],axis=1)
bday_update

,player_name,team_abbreviation_y,draft_year_y,gp_y,pts_y,reb_y,ast_y,Key_y,Injuries_y,TeamId_y,PlayerId_y,BIRTHDATE_y
0,Marcus Camby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kevin Garnett,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Juwan Howard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kurt Thomas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kobe Bryant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
171,Mamadi Diakite,MIL,2020.0,14.0,3.1,2.4,0.6,Mamadi DiakiteMIL2020,0.0,1.610613e+09,NaN,1/21/1997
172,Markus Howard,DEN,2020.0,37.0,2.8,0.6,0.5,Markus HowardDEN2020,0.0,1.610613e+09,NaN,3/3/1999
173,Marques Bolden,CLE,2019.0,1.0,0.0,2.0,0.0,Marques BoldenCLE2019,0.0,1.610613e+09,NaN,4/17/1998
174,Marques Bolden,CLE,2020.0,6.0,1.2,1.0,0.0,Marques BoldenCLE2020,0.0,1.610613e+09,NaN,4/17/1998


In [193]:
bday_update.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,Age,PlayerId,BIRTHDATE
0,Marcus Morris,HOU,2011,17,2.4,0.9,0.2,Marcus MorrisHOU2011,0.0,1610612745,NaN,202694,1989-09-02T00:00:00
1,Marcus Morris,PHX,2012,77,7.7,3.6,0.8,Marcus MorrisPHX2012,0.0,1610612756,NaN,202694,1989-09-02T00:00:00
2,Marcus Morris,PHX,2013,82,9.7,3.9,1.1,Marcus MorrisPHX2013,0.0,1610612756,NaN,202694,1989-09-02T00:00:00
3,Marcus Morris,PHX,2014,81,10.4,4.8,1.6,Marcus MorrisPHX2014,0.0,1610612756,NaN,202694,1989-09-02T00:00:00
4,Marcus Morris,DET,2015,80,14.1,5.1,2.5,Marcus MorrisDET2015,0.0,1610612765,NaN,202694,1989-09-02T00:00:00


In [192]:
bday_update=bday_update.rename(columns = {'BIRTHDATE_y':'BIRTHDATE'})

In [194]:
injuries_all_up.columns

Index(['player_name', 'team_abbreviation', 'draft_year', 'gp', 'pts', 'reb',
       'ast', 'Key', 'Injuries', 'PlayerId', 'TeamId', 'ID', 'BIRTHDATE',
       'Age'],
      dtype='object')

In [203]:
injuries_all_up2.shape

(5483,)

In [216]:
injuries_all_up2 = injuries_all_up.merge(bday_update[['Key','PlayerId','BIRTHDATE']],how='left',on='Key')
injuries_all_up2.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId_x,TeamId,ID,BIRTHDATE_x,Age,PlayerId_y,BIRTHDATE_y
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,201589,1988-03-25,33.0,NaN,NaN
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,201985,1986-10-07,34.0,NaN,NaN
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,2230,1610612764,2230,1978-11-08,42.0,NaN,NaN
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,200772,1984-02-09,37.0,NaN,NaN
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,2679,1610612766,2679,1980-08-28,41.0,NaN,NaN


In [233]:
for i,row in injuries_all_up2.iterrows():
    if row['PlayerId_x']=='nan':
        pid = row['PlayerId_y']
        injuries_all_up2.loc[i,'PlayerId_x']=pid
        
   # if pd.isnull(row['BIRTHDATE_x']):
   #     bday = row['BIRTHDATE_y']
   #     injuries_all_up2.loc[i,'BIRTHDATE_x']=bday

injuries_all_up2.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId_x,TeamId,ID,BIRTHDATE_x,Age,PlayerId_y,BIRTHDATE_y
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,201589,1988-03-25 00:00:00,33.0,NaN,NaN
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,201985,1986-10-07 00:00:00,34.0,NaN,NaN
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,2230,1610612764,2230,1978-11-08 00:00:00,42.0,NaN,NaN
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,200772,1984-02-09 00:00:00,37.0,NaN,NaN
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,2679,1610612766,2679,1980-08-28 00:00:00,41.0,NaN,NaN


In [236]:
null_b=injuries_all_up2[injuries_all_up2['BIRTHDATE_x'].isnull()]

In [238]:
null_b.to_csv('../Data/nullbirthdays.csv',index=False)

In [239]:
playersAttr_df[playersAttr_df['DISPLAY_FIRST_LAST']=='Michael Frazier II']

,ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PTS,AST,REB,ALL_STAR_APPEARANCES,PIE
1297,1626187,Michael,Frazier II,Michael Frazier II,"Frazier II, Michael",M. Frazier II,michael-frazier-ii,1994-03-08T00:00:00,Florida,USA,...,Y,Y,Undrafted,Undrafted,Undrafted,2.1,0.2,0.8,0.0,NaN


In [242]:
null_b=null_b.merge(playersAttr_df[['ID','DISPLAY_FIRST_LAST','BIRTHDATE']],how="left",left_on='player_name',right_on='DISPLAY_FIRST_LAST')
null_b.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId_x,TeamId,ID_x,BIRTHDATE_x,Age,PlayerId_y,BIRTHDATE_y,ID_y,DISPLAY_FIRST_LAST,BIRTHDATE
0,Marcus Camby,POR,2010,59,4.7,10.3,2.1,Marcus CambyPOR2010,0.0,NaN,1610612757,NaN,NaT,NaN,NaN,NaN,948,Marcus Camby,1974-03-22T00:00:00
1,Kevin Garnett,BOS,2010,71,14.9,8.9,2.4,Kevin GarnettBOS2010,1.0,NaN,1610612738,NaN,NaT,NaN,NaN,NaN,708,Kevin Garnett,1976-05-19T00:00:00
2,Juwan Howard,MIA,2010,57,2.4,2.1,0.4,Juwan HowardMIA2010,3.0,NaN,1610612748,NaN,NaT,NaN,NaN,NaN,436,Juwan Howard,1973-02-07T00:00:00
3,Kurt Thomas,CHI,2010,52,4.1,5.8,1.2,Kurt ThomasCHI2010,4.0,NaN,1610612741,NaN,NaT,NaN,NaN,NaN,703,Kurt Thomas,1972-10-04T00:00:00
4,Kobe Bryant,LAL,2010,82,25.3,5.1,4.7,Kobe BryantLAL2010,0.0,NaN,1610612747,NaN,NaT,NaN,NaN,NaN,977,Kobe Bryant,1978-08-23T00:00:00


In [247]:
null_b.columns

Index(['player_name', 'team_abbreviation', 'draft_year', 'gp', 'pts', 'reb',
       'ast', 'Key', 'Injuries', 'PlayerId_x', 'TeamId', 'ID_x', 'Age',
       'PlayerId_y', 'BIRTHDATE_y', 'ID_y', 'BIRTHDATE'],
      dtype='object')

In [246]:
null_b=null_b.drop(['BIRTHDATE_x','DISPLAY_FIRST_LAST'],axis=1)

In [263]:
null_b.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,TeamId,ID_y,BIRTHDATE
0,Marcus Camby,POR,2010,59,4.7,10.3,2.1,Marcus CambyPOR2010,0.0,1610612757,948,1974-03-22T00:00:00
1,Kevin Garnett,BOS,2010,71,14.9,8.9,2.4,Kevin GarnettBOS2010,1.0,1610612738,708,1976-05-19T00:00:00
2,Juwan Howard,MIA,2010,57,2.4,2.1,0.4,Juwan HowardMIA2010,3.0,1610612748,436,1973-02-07T00:00:00
3,Kurt Thomas,CHI,2010,52,4.1,5.8,1.2,Kurt ThomasCHI2010,4.0,1610612741,703,1972-10-04T00:00:00
4,Kobe Bryant,LAL,2010,82,25.3,5.1,4.7,Kobe BryantLAL2010,0.0,1610612747,977,1978-08-23T00:00:00


In [257]:
null_b[null_b['ID_x'].notnull()]

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId_x,TeamId,ID_x,Age,PlayerId_y,BIRTHDATE_y,ID_y,BIRTHDATE


In [262]:
null_b.isna().sum()

player_name          0
team_abbreviation    0
draft_year           0
gp                   0
pts                  0
reb                  0
ast                  0
Key                  0
Injuries             0
TeamId               0
ID_y                 2
BIRTHDATE            2
dtype: int64

In [261]:
null_b=null_b.drop(['PlayerId_x','ID_x','PlayerId_y','BIRTHDATE_y','Age'],axis=1)

In [264]:
injuries_all_up3=injuries_all_up2.merge(null_b[['Key','BIRTHDATE','ID_y']],how='left',on='Key')
injuries_all_up3.head()

,player_name,team_abbreviation,draft_year,gp,pts,reb,ast,Key,Injuries,PlayerId_x,TeamId,ID,BIRTHDATE_x,Age,PlayerId_y,BIRTHDATE_y,BIRTHDATE,ID_y
0,Darrell Arthur,MEM,2010,80,9.1,4.3,0.7,Darrell ArthurMEM2010,1.0,201589,1610612763,201589,1988-03-25 00:00:00,33.0,NaN,NaN,NaN,NaN
1,AJ Price,IND,2010,50,6.5,1.4,2.2,AJ PriceIND2010,4.0,201985,1610612754,201985,1986-10-07 00:00:00,34.0,NaN,NaN,NaN,NaN
2,Maurice Evans,WAS,2010,73,6.3,2.1,0.6,Maurice EvansWAS2010,0.0,2230,1610612764,2230,1978-11-08 00:00:00,42.0,NaN,NaN,NaN,NaN
3,Maurice Ager,MIN,2010,4,3.8,0.5,0.3,Maurice AgerMIN2010,2.0,200772,1610612750,200772,1984-02-09 00:00:00,37.0,NaN,NaN,NaN,NaN
4,Matt Carroll,CHA,2010,54,4.4,1.3,0.4,Matt CarrollCHA2010,0.0,2679,1610612766,2679,1980-08-28 00:00:00,41.0,NaN,NaN,NaN,NaN


In [265]:
injuries_all_up3.isna().sum()

player_name             0
team_abbreviation       0
draft_year              0
gp                      0
pts                     0
reb                     0
ast                     0
Key                     0
Injuries                0
PlayerId_x            169
TeamId                 92
ID                    176
BIRTHDATE_x            58
Age                   176
PlayerId_y           5476
BIRTHDATE_y          5365
BIRTHDATE            5427
ID_y                 5427
dtype: int64

In [266]:
injuries_all_up3.shape

(5483, 18)

In [268]:
injuries_all_up3.to_csv('../Data/Injuries_all_up.csv',index=True)

## Did some final clean-up in excel and renamed the file to 'Games and injuries stats-2010-2020.csv'